## Paso 1: Instalación de dependencias y configuración del entorno

In [1]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 74kB/s 
     |████████████████████████████████| 419kB 36.7MB/s 
     |████████████████████████████████| 3.0MB 37.7MB/s 


## Paso 2: Importar las dependencias del proyecto

In [0]:
import time
import numpy as np
import tensorflow as tf

In [3]:
tf.__version__

'2.0.0-alpha0'

## Paso 3: Pre procesado del dataset

### Cargar el dataset del MNIST

In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


### Normalización de las imágenes

In [0]:
X_train = X_train / 255.
X_test = X_test / 255.

In [6]:
X_train.shape

(60000, 28, 28)

### Redimensionar el dataset

In [0]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [8]:
X_train.shape

(60000, 784)

## Paso 4: Entrenamiento distribuido

### Definir un modelo normal (no distribuido)

In [0]:
model_normal = tf.keras.models.Sequential()

In [0]:
model_normal.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))

In [0]:
model_normal.add(tf.keras.layers.Dropout(rate=0.2))

In [0]:
model_normal.add(tf.keras.layers.Dense(units=10, activation='softmax'))

In [0]:
model_normal.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Definir una estrategia distribuida

In [0]:
distribute = tf.distribute.MirroredStrategy()

### Definir un modelo distribuido

In [0]:
with distribute.scope():
  model_distributed = tf.keras.models.Sequential()
  model_distributed.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
  model_distributed.add(tf.keras.layers.Dropout(rate=0.2))
  model_distributed.add(tf.keras.layers.Dense(units=10, activation='softmax'))
  model_distributed.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Comparar la velocidad de entrenamiento normal vs distribuida

In [16]:
start_time = time.time()
model_distributed.fit(X_train, y_train, epochs=10, batch_size=25)
print("El entrenamiento distribuido ha tardado: {}".format(time.time() - start_time))

Epoch 1/10
2400/2400 [==============================] - 16s 6ms/step - loss: 0.2813 - sparse_categorical_accuracy: 0.9178
Epoch 2/10
2400/2400 [==============================] - 15s 6ms/step - loss: 0.1362 - sparse_categorical_accuracy: 0.9597
Epoch 3/10
2400/2400 [==============================] - 15s 6ms/step - loss: 0.1035 - sparse_categorical_accuracy: 0.9683
Epoch 4/10
2400/2400 [==============================] - 15s 6ms/step - loss: 0.0852 - sparse_categorical_accuracy: 0.9739
Epoch 5/10
2400/2400 [==============================] - 15s 6ms/step - loss: 0.0730 - sparse_categorical_accuracy: 0.9769
Epoch 6/10
2400/2400 [==============================] - 15s 6ms/step - loss: 0.0651 - sparse_categorical_accuracy: 0.9790
Epoch 7/10
2400/2400 [==============================] - 15s 6ms/step - loss: 0.0576 - sparse_categorical_accuracy: 0.9814
Epoch 8/10
2400/2400 [==============================] - 15s 6ms/step - loss: 0.0518 - sparse_categorical_accuracy: 0.9829
Epoch 9/10
2400/2400 [==

In [17]:
start_time = time.time()
model_normal.fit(X_train, y_train, epochs=10, batch_size=25)
print("El entrenamiento normal ha tardado: {}".format(time.time() - start_time))

Epoch 1/10
60000/60000 [==============================] - 12s 202us/sample - loss: 0.2892 - sparse_categorical_accuracy: 0.9165
Epoch 2/10
60000/60000 [==============================] - 12s 197us/sample - loss: 0.1401 - sparse_categorical_accuracy: 0.9575
Epoch 3/10
60000/60000 [==============================] - 12s 197us/sample - loss: 0.1075 - sparse_categorical_accuracy: 0.9674
Epoch 4/10
60000/60000 [==============================] - 12s 196us/sample - loss: 0.0873 - sparse_categorical_accuracy: 0.9728
Epoch 5/10
60000/60000 [==============================] - 12s 196us/sample - loss: 0.0734 - sparse_categorical_accuracy: 0.9764
Epoch 6/10
60000/60000 [==============================] - 12s 197us/sample - loss: 0.0653 - sparse_categorical_accuracy: 0.9791
Epoch 7/10
60000/60000 [==============================] - 12s 195us/sample - loss: 0.0579 - sparse_categorical_accuracy: 0.9816
Epoch 8/10
60000/60000 [==============================] - 12s 195us/sample - loss: 0.0521 - sparse_categ